
# WAV 꼬리 구간을 MP4와 싱크하여 병합하기 (Jupyter 버전)

이 노트북은 다음을 수행합니다.

1. WAV 디렉토리와 MP4 디렉토리를 지정합니다.  
2. 각 MP4 길이에 맞춰 **WAV의 맨 뒷부분(꼬리)**에서 동일 길이만큼 잘라,  
3. 해당 오디오를 영상에 붙여 `_synced.mp4`를 생성합니다.  
4. 결과는 **원래 MP4 디렉토리**에 저장됩니다.

**전제**: 시스템에 `ffmpeg`, `ffprobe`가 설치되어 있어야 합니다.


In [33]:
# 셀 1: import 및 유틸 함수 정의
from pathlib import Path
import subprocess

def run(cmd):
    """ffmpeg/ffprobe 실행 유틸리티"""
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if p.returncode != 0:
        raise RuntimeError(f"Command failed: {' '.join(cmd)}\nSTDERR:\n{p.stderr}")
    return p.stdout.strip()

def probe_duration(path: Path) -> float:
    """ffprobe로 미디어 길이(초) 가져오기"""
    out = run([
        "ffprobe", "-v", "error", "-show_entries", "format=duration",
        "-of", "default=nw=1:nk=1", str(path)
    ])
    return float(out)


In [ ]:
# 셀 2: 경로 지정 (직접 파일 지정)
WAV_PATH = Path("./data/Lieu/aud.wav")        # 오디오 파일 경로
MP4_PATH = Path("./output/Lieu/test/gt/renders/out.mp4")  # 영상 파일 경로

assert WAV_PATH.exists(), f"WAV 파일을 찾을 수 없습니다: {WAV_PATH}"
assert MP4_PATH.exists(), f"MP4 파일을 찾을 수 없습니다: {MP4_PATH}"

AUDIO_BITRATE = "192k"  # 출력 오디오 비트레이트
OUTPUT_PATH = MP4_PATH.with_name(f"{MP4_PATH.stem}_audio.mp4")


In [35]:
# 셀 3: 길이 계산 및 싱크 시점 산출
wav_dur = probe_duration(WAV_PATH)
vid_dur = probe_duration(MP4_PATH)

# WAV의 끝부분에서 영상 길이만큼만 자름
start_time = max(0.0, wav_dur - vid_dur)

print(f"WAV 길이: {wav_dur:.2f}s, MP4 길이: {vid_dur:.2f}s")
print(f"오디오 시작 시점 (tail): {start_time:.2f}s")


WAV 길이: 281.28s, MP4 길이: 12.00s
오디오 시작 시점 (tail): 269.28s


In [36]:
# 셀 4: ffmpeg 명령 생성 및 실행
# 오디오 tail을 잘라 붙여 싱크된 영상 생성
cmd = [
    "ffmpeg", "-y",
    "-i", str(MP4_PATH),
    "-ss", f"{start_time:.3f}", "-t", f"{vid_dur:.3f}", "-i", str(WAV_PATH),
    "-map", "0:v:0", "-map", "1:a:0",
    "-c:v", "copy", "-c:a", "aac", "-b:a", AUDIO_BITRATE,
    "-shortest",
    str(OUTPUT_PATH)
]

print(" ".join(cmd))  # 실행 전 명령 확인

run(cmd)
print(f"[완료] 싱크된 영상 저장됨: {OUTPUT_PATH}")

ffmpeg -y -i output/Lieu_1_frames_02/test/ours_None/renders/out.mp4 -ss 269.280 -t 12.000 -i data/Lieu/aud.wav -map 0:v:0 -map 1:a:0 -c:v copy -c:a aac -b:a 192k -shortest output/Lieu_1_frames_02/test/ours_None/renders/out_audio.mp4
[완료] 싱크된 영상 저장됨: output/Lieu_1_frames_02/test/ours_None/renders/out_audio.mp4



## 참고 & 디버깅 팁
- **동일 이름 매칭 권장**: `clip001.mp4` ↔ `clip001.wav` 형태가 가장 안전합니다.  
- **WAV가 하나뿐이면** 모든 MP4에 대해 그 WAV의 tail을 각자 잘라 붙입니다.  
- **영상이 오디오보다 긴 경우**: `USE_APAD=True`로 무음 패딩을 적용할 수 있습니다.  
- `DRY_RUN=True`로 명령만 출력해보고, 정상임을 확인한 다음 `False`로 실행하세요.
